In [3]:
import pandas
import csv
import stanza
import pymorphy2
import numpy as np
import json
import random
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from string import punctuation
from string import digits
from pymystem3 import Mystem
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

stanza.download('ru')
nlp = stanza.Pipeline('ru')
morph = pymorphy2.MorphAnalyzer()
m = Mystem()

2022-10-13 17:10:25 INFO: Downloading default packages for language: ru (Russian) ...


2022-10-13 17:11:42 INFO: Finished downloading models and saved to C:\Users\katia\stanza_resources.
2022-10-13 17:11:42 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-10-13 17:11:46 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2022-10-13 17:11:46 INFO: Use device: cpu
2022-10-13 17:11:46 INFO: Loading: tokenize
2022-10-13 17:11:46 INFO: Loading: pos
2022-10-13 17:11:47 INFO: Loading: lemma
2022-10-13 17:11:47 INFO: Loading: depparse
2022-10-13 17:11:47 INFO: Loading: ner
2022-10-13 17:11:50 INFO: Done loading processors!


In [123]:
punctuation += '—' + '»' + '«'
punctuation = punctuation.split('-')[0] + punctuation.split('-')[1]

#### Создание, разметка корпуса и объяснение того, почему этот текст подходит для оценки:

Я выбрала цикл «лингвистических сказок» Л. Петрушевской "Пуськи бятые", так все неслужебные слова в нем полностью выдуманы, но сохраняют некоторые признаки изначальных частей речи (окончания, суффиксы). Таким образом мы можем проверить, насколько автоматический теггер справится с синтаксически верными последовательностями ненастоящих слов (авторских неологизмов).

#### Какой тегсет я беру для разметки и почему:

Изначально я выбрала для разметки UD, так как он, как мне показалось, наименее специфичный (а также самый привычный мне...)
При приведении тегов к одинаковому набору я упростила сет до:
* _NOUN_ (существительные, в т.ч. _PROPN_), 
* _ADJ_ (краткие и полные прилагательные, причастия), 
* _VERB_ (глаголы),
* _ADP_ (предлоги),
* _CONJ_ (как подчинительные, так и сочинительные союзы),
* _INTJ_ (междометия),
* _PART_ (частицы),
* _ADV_ (наречия, деепричастия),
* _PRON_ (все виды местоимений),
* _X_ (звукоподражания, иностранные куски)

Это было сделано с той целью, что, в первую очередь, мы хотим не просто точность для отдельных сложных слов, а общую способность теггера справиться с текстом из неологизмов.

Я отношу причастия к прилагательным, а деепричастия к наречиям из-за того, что по форме они напоминают друг друга. В условиях текста, где все слова неологизмы, это, как мне кажется, самое важное.

In [61]:
with open("pus'ki_b'atyje.txt", 'r', encoding='utf-8') as file:
    text = file.read()

In [62]:
text_tokenized = word_tokenize(text)

#### Три POS теггера для русского языка:

* __Stanza__

In [32]:
stanza_text = nlp(text)
with open("stanza_pus'ki_b'atyje.txt", 'w', encoding='utf-8') as stanza_file:
    for sent in stanza_text.sentences:
        for word in sent.words:
            stanza_file.write(f'{word.text}\t{word.upos}\n')

* __PyMorphy__

In [44]:
with open("pymorphy_pus'ki_b'atyje.txt", 'w', encoding='utf-8') as pymorphy_file:
    for word in text_tokenized:
        pymorphy_file.write(f'{word}\t{morph.parse(word)[0].tag.POS}\n')

* __MyStem__ 

(господи, какой он медленный, зашто)

In [86]:
with open("mystem_pus'ki_b'atyje.txt", 'w', encoding='utf-8') as mystem_file:
    for word in text_tokenized:
        stems = m.analyze(word)
        for item in stems:
            if 'analysis' in item:
                if item['analysis'] and 'gr' in item['analysis'][0]:
                    mystem_file.write(f"{item['text']}\t{item['analysis'][0]['gr'].split(',')[0].split('=')[0]}\n")
                else:
                    mystem_file.write(f"{item['text']}\tX\n")

Здесь я, как раз, смотрела, какие вообще встречаются теги в каждом из наборов:

In [207]:
def what_are_your_tags(name):
    pos_tags = []
    with open(name, encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if len(line.split('\t')) > 1:
                tag = line.split('\t')[1]
                if tag not in pos_tags:
                    pos_tags.append(tag)
    return pos_tags

In [208]:
pos_tag_manual = what_are_your_tags("processed_pus'ki_b'atyje.txt")
pos_tag_stanza = what_are_your_tags("stanza_pus'ki_b'atyje.txt")
pos_tag_pymorphy = what_are_your_tags("pymorphy_pus'ki_b'atyje.txt")
pos_tag_mystem = what_are_your_tags("mystem_pus'ki_b'atyje.txt")

Встретившиеся теги для каждого из POS теггеров:

In [209]:
print(f'Manual POS tags: {pos_tag_manual}\nStanza POS tags: {pos_tag_stanza}\nPyMorphy POS tags: {pos_tag_pymorphy}\nMyStem POS tags: {pos_tag_mystem}')

Manual POS tags: ['NOUN', 'ADJ', 'NUM', 'PUNCT', 'VERB', 'PROPN', 'ADP', 'CCONJ', 'INTJ', 'PART', 'ADV', 'PRON', 'X', 'SCONJ', 'AUX', 'SYM']
Stanza POS tags: ['NOUN', 'ADJ', 'NUM', 'PUNCT', 'PROPN', 'ADP', 'CCONJ', 'VERB', 'INTJ', 'PART', 'SCONJ', 'ADV', 'X', 'AUX', 'SYM', 'PRON']
PyMorphy POS tags: ['NOUN', 'ADJF', 'None', 'VERB', 'PREP', 'CONJ', 'ADJS', 'PRCL', 'PRTF', 'INFN', 'INTJ', 'ADVB', 'GRND', 'PRTS']
MyStem POS tags: ['S', 'V', 'PR', 'CONJ', 'PART', 'X', 'A', 'ADVPRO', 'INTJ', 'ADV', 'SPRO', 'APRO']


#### Cведение к единому стандарту

_См. про упрощение тегов выше:_

* __Stanza и размеченный вручную__ (оба опираются на Universal Dependencies)

In [194]:
def from_ud(text):
    pos_set = []
    for line in text:
        line = line.strip()
        if len(line.split('\t')) > 1:
            word = line.split('\t')[0].strip(punctuation)
            tag = line.split('\t')[1]
            if tag == 'PROPN':
                tag = 'NOUN'
            elif tag.endswith('CONJ'):
                tag = 'CONJ'
            elif tag == 'PUNCT':
                continue
            elif tag == 'SYM':
                continue
            elif tag == 'AUX':
                tag = 'VERB'
            if word and not word.isdigit():
                pos_set.append([word, tag])
    return pos_set

* __PyMorphy__

In [105]:
def from_pymorphy(text):
    pos_set = []
    for line in text:
        line = line.strip()
        if len(line.split('\t')) > 1:
            word = line.split('\t')[0].strip(punctuation)
            tag = line.split('\t')[1]
            if tag.startswith('ADJ') or tag.startswith('PRT'):
                tag = 'ADJ'
            elif tag == 'ADVB' or tag == 'GRND':
                tag = 'ADV'
            elif tag == 'INFN':
                tag = 'VERB'
            elif tag == 'None':
                tag = 'X'
            elif tag == 'PREP':
                tag = 'ADP'
            elif tag == 'PRCL':
                tag = 'PART'
            if word and not word.isdigit():
                pos_set.append([word, tag])
    return pos_set

* __MyStem__

In [122]:
def from_mystem(text):
    pos_set = []
    for line in text:
        line = line.strip()
        if len(line.split('\t')) > 1:
            word = line.split('\t')[0].strip(punctuation)
            tag = line.split('\t')[1]
            if tag.endswith('PRO'):
                tag = 'PRON'
            elif tag == 'A':
                tag = 'ADJ'
            elif tag == 'S':
                tag = 'NOUN'
            elif tag == 'PR':
                tag = 'ADP'
            elif tag == 'V':
                tag = 'VERB'
            if word and not word.isdigit():
                pos_set.append([word, tag])
    return pos_set

In [195]:
with open("pymorphy_pus'ki_b'atyje.txt", encoding='utf-8') as file:
    file = file.readlines()
    set_pymorphy = from_pymorphy(file)
    
with open("mystem_pus'ki_b'atyje.txt", encoding='utf-8') as file:
    file = file.readlines()
    set_mystem = from_mystem(file)

with open("stanza_pus'ki_b'atyje.txt", encoding='utf-8') as file:
    file = file.readlines()
    set_stanza = from_ud(file)
    
with open("processed_pus'ki_b'atyje.txt", encoding='utf-8') as file:
    file = file.readlines()
    set_manual = from_ud(file)

#### Сравниваем с размеченным руками эталоном:

In [196]:
def wrong_or_right(set_pairs):
    if len(set_pairs) > len(set_manual):
        bigger_set = set_pairs
        set_pairs = set_manual
    else:
        bigger_set = set_manual
    wrong = 0
    right = 0
    for index, pair in enumerate(set_pairs):
        for i in range(index, len(bigger_set)):
            if bigger_set[i][0] == pair[0]:
                if bigger_set[i][1] == pair[1]:
                    right += 1
                else:
                    wrong += 1
                break
    return wrong, right

In [197]:
wrong_pymorphy, right_pymorphy = wrong_or_right(set_pymorphy)
wrong_stanza, right_stanza = wrong_or_right(set_stanza)
wrong_mystem, right_mystem = wrong_or_right(set_mystem)

#### Считаем _accuracy_:

In [198]:
print(f'MyStem: accuracy = {right_mystem / (wrong_mystem + right_mystem)}\nPyMorphy: accuracy = {right_pymorphy / (wrong_pymorphy + right_pymorphy)}\nStanza: accuracy = {right_stanza / (wrong_stanza + right_stanza)}')

MyStem: accuracy = 0.6646586345381527
PyMorphy: accuracy = 0.7049626104690686
Stanza: accuracy = 0.8448275862068966


Лучше всего с задачей справилась **_Stanza_**

In [1]:
with open("good_reviews.txt", 'r', encoding='utf-8') as file:
    text_review_good = file.read()
    
with open("bad_reviews.txt", 'r', encoding='utf-8') as file:
    text_review_bad = file.read()

In [5]:
stanza_text_review_good = nlp(text_review_good)
stanza_text_review_bad = nlp(text_review_bad)

In [102]:
with open('good_pos.txt', 'w', encoding='utf-8') as f:
    for sent in stanza_text_review_good.sentences:
        for word in sent.words:
            f.write(f'{word.text}\t{word.lemma}\t{word.upos}\n')

In [103]:
with open('bad_pos.txt', 'w', encoding='utf-8') as f:
    for sent in stanza_text_review_bad.sentences:
        for word in sent.words:
            f.write(f'{word.text}\t{word.lemma}\t{word.upos}\n')

#### 3 шаблона:
1. _вообще не, совсем не_ + NOUN / VERB / ADJ → скорее негативное (_вообще не понравилось_, _совсем не перспективный_, _вообще не айс_)
2. _вполне_ + ADV / ADJ → скорее положительное (_вполне хорошо_, _вполне достойный_)
3. _никакой_ (в разных падежах) + NOUN → дополнительная дистрибуция (_никаких проблем_ vs. _никакой перспективы_)

#### Сам чанкер:

Из него достаем те n-граммы, которые встречаются только в плохих / хороших ревью.

In [140]:
def chunker(text_name):
    first_pattern = []
    second_pattern = []
    third_pattern = []
    tagged_lines = []
    with open(text_name, encoding='utf-8') as tagged_file:
        for line in tagged_file:
            if len(line.split('\t')) == 3:
                word, lemma, pos_tag = line.strip().split('\t')
                if pos_tag == 'PUNCT':
                    continue
                tagged_lines.append([word, lemma, pos_tag])
    for index, line in enumerate(tagged_lines):
        if len(tagged_lines) > index + 2:
            word, lemma, pos_tag = line
            word_next, lemma_next, pos_tag_next = tagged_lines[index + 1]
            word_next_next, lemma_next_next, pos_tag_next_next = tagged_lines[index + 2]
            if (word.lower() == 'вообще' or word.lower() == 'совсем') and word_next == 'не' and (pos_tag_next_next in ['NOUN', 'VERB', 'ADJ']):
                if lemma + ' ' + lemma_next + ' ' + lemma_next_next not in first_pattern:
                    first_pattern.append(lemma + ' ' + lemma_next + ' ' + lemma_next_next)
            elif word == 'вполне' and (pos_tag_next == 'ADV' or pos_tag_next == 'ADJ'):
                if lemma + ' ' + lemma_next not in second_pattern:
                    second_pattern.append(lemma + ' ' + lemma_next)
            elif lemma == 'никакой' and pos_tag_next == 'NOUN':
                if lemma + ' ' + lemma_next not in third_pattern:
                    third_pattern.append(lemma + ' ' + lemma_next)
    return set(first_pattern), set(second_pattern), set(third_pattern)

In [142]:
only_bad_grams = []
for i, chunk in enumerate(chunker('bad_pos.txt')):
    only_bad_grams.append(chunk.difference(chunker('good_pos.txt')[i]))

In [145]:
only_good_grams = []
for i, chunk in enumerate(chunker('good_pos.txt')):
    only_good_grams.append(chunk.difference(chunker('bad_pos.txt')[i]))

#### Встраивание функции в программу из предыдущей домашки:

In [110]:
with open('result.json', encoding='utf-8') as json_file:
    db_mts = json.load(json_file)

In [112]:
def splitter(db):  # to divide into bad reviews and good ones
    goods = []
    bads = []
    for item in db:
        items = list(map(int, db[item][-1].values()))  # grades into integers
        average = sum(items) / len(items)  # average grade for all the criteria
        if average > 3:  # more than 3 -> good
            text = db_mts[item][0] + '\n' + db[item][1]
            goods.append(text)
        elif average < 3:  # less than 3 -> bad
            text = db_mts[item][0] + '\n' + db[item][1]
            bads.append(text)
    if len(goods) > len(bads):  # make them equal
        goods = goods[:len(bads)]
    else:
        bads = bads[:len(goods)]
    with open('good_reviews.txt', 'w', encoding='utf-8') as f_good:
        f_good.write('\n'.join(goods))
    with open('bad_reviews.txt', 'w', encoding='utf-8') as f_bad:
        f_bad.write('\n'.join(bads))
    return goods, bads

In [130]:
def checker(word):  # checker for digits and punctuation
    for char in punctuation:
        if char in word:
            return False
    for char in digits:
        if char in word:
            return False
    return True

In [131]:
def tokenizer(text):  # tokenizer...
    tokenized = []
    for part in text:
        words = []
        tokenized_part = word_tokenize(part)  # tokenize
        for word in tokenized_part:
            if checker(word):
                word = word.lower()  # make lower
                word = morph.parse(word)[0].normal_form  # to normal form
                words.append(word)
        tokenized.append(words)
    return tokenized

In [147]:
def create_dic(tokenized_text, good_or_bad):  # frequency dictionary of the tokens
    good_tokens = dict()
    bad_tokens = dict()
    for index, part in enumerate(tokenized_text):
        for token in part:
            if good_or_bad[index] == 1:
                if token not in good_tokens:
                    good_tokens[token] = 1
                else:
                    good_tokens[token] += 1
            else:
                if token not in bad_tokens:
                    bad_tokens[token] = 1
                else:
                    bad_tokens[token] += 1
    good_tokens = dict(sorted(good_tokens.items(), key=lambda item: item[1], reverse=True))  # good dictionary sorting
    bad_tokens = dict(sorted(bad_tokens.items(), key=lambda item: item[1], reverse=True))  # bad dictionary sorting
    good_vals = set(good_tokens.keys())  # all the good tokens
    bad_vals = set(bad_tokens.keys())  # # all the bad tokens
    return good_tokens, bad_tokens, good_vals, bad_vals

In [151]:
def del_small(words, dictionary):  # delete non-frequent tokens
    real_words = []
    for word in words:
        if dictionary[word] > 2:  # occurrence is more than 2
            real_words.append(word)
    return real_words

In [132]:
good_reviews, bad_reviews = splitter(db_mts)

In [173]:
X = good_reviews + bad_reviews
y = np.concatenate((np.full((len(good_reviews),), 1), np.full((len(bad_reviews),), 0)))

In [174]:
X = tokenizer(X)

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

In [148]:
good_dic, bad_dic, good_words, bad_words = create_dic(X_train, y_train)

In [149]:
only_good = good_words.difference(bad_words)
only_bad = bad_words.difference(good_words)

In [152]:
really_bad = del_small(only_bad, bad_dic)
really_good = del_small(only_good, good_dic)

Меняем функцию *good_or_bad_checker()* с n-граммами:

(я еще добавила веса, так как подумала, что такие сочетания точно больше говорят об отзыве, чем 1 слово)

In [176]:
def good_or_bad_checker(test, good, bad):
    preds = []
    for review in test:
        for_good = 0
        for_bad = 0
        for i, token in enumerate(review):
            if token in good:
                for_good += 1
            elif token in bad:
                for_bad += 1
            if i + 1 < len(review):
                if token + ' ' + review[i + 1] in only_good_grams[1] or token + ' ' + review[i + 1] in only_good_grams[2]:
                    for_good += 2
                elif token + ' ' + review[i + 1] in only_bad_grams[1] or token + ' ' + review[i + 1] in only_bad_grams[2]:
                    for_bad += 2
            if i + 2 < len(review):
                if token + ' ' + review[i + 1] + ' ' + review[i + 2] in only_good_grams[0]:
                    for_good += 2
                elif token + ' ' + review[i + 1] + ' ' + review[i + 2] in only_bad_grams[0]:
                    for_bad += 2
        if for_good > for_bad:
            preds.append(1)
        elif for_bad > for_good:
            preds.append(0)
        else:
            preds.append(random.randint(0, 1))
    return preds

#### Считаем _accuracy_:

In [177]:
y_preds = good_or_bad_checker(X_test, really_good, really_bad)

In [179]:
accuracy_score(y_test, y_preds)

0.7589285714285714

В этот раз качество **лучше почти на .02** (по сравнению с 0.7410714285714286 из прошлого ДЗ)!